Validación con datos generados sintéticamente
=============================================

No siempre es sencillo obtner acceso a repositorios de datos que son lo suficientemente grandes y con predictores realistas como ser nombre, edad, dirección, fecha de nacimiento, sexo, etc. Estos conjuntos de datos no son solo útiles para aquellas personas que desean probar diferentes técnicas o modelos de aprendizaje automático, sino que también son muy interesantes para validadores de software (testers) que les gustaría tener un conjuntos de datos arbitrariamente grandes (aunque con datos ficticios) sobre los cuales ejecutar pruebas de calidad de los modelos de aprendizaje que otros desarrolladores crearon. Esto les facilita la posibilidad de probar determinados casos de borde sin necesidad de modificar datos manualmente ni tener que crear de forma artesanal conjuntos de datos para las validaciones

## Faker

`Faker` es una libería de Python que permite generar conjuntos de datos sintéticos. `Faker` no solo permite la generación de tipos de datos simples, sino que también puede generar datos como nombres, direcciones, teléfonos o direcciones de correo electrónico. Esta información la puede generar para diferentes `locations` incluyendo el idioma Español. Lamentablemente, no dispone de una implementación específica para Argentina, siendo las variaciones más aproximadas las de Mexico (es-MX) o España (es-ES). `Faker` es extensible, lo que significa que si existe un tipo de dato que no es soportado actualmente, se lo puede implementar para que la librería lo genere. Es ampliamente utilizado por la comunida de código abierto por lo que existen multiples extensiones ya creadas.

Para más información sobre `Faker` recomendamos revisar la documentación oficial en https://faker.readthedocs.io/en/latest

### Instalación

En Linux o Windows, podemos instalar la libreria via `pip`:

In [ ]:
!pip install Faker

In [2]:
from faker import Faker

In [14]:
fake = Faker('es-ES')

In [27]:
fake.pyint(min_value=0, max_value=999, step=5)

760

## Ejemplo

Para demostrar como realizar validaciones utilizando conjunto de datos sintéticos, trabajaremos con el conjunto de datos de 'Pima Indians Diabetes Database' (https://www.openml.org/d/37), proveniente originalmente del National Institute of Diabetes and Digestive and Kidney Diseases. El objetivo del conjunto de datos es el de predecir si un paciente tiene o no diabetes, basado en ciertas mediciones que se incluyen. Este conjunto es interesante para esta demostración porque tiene varias restricciones en cuanto a como se recolectaron los datos. En particular, todos los pacientes son de sexo femenino, de al menos 21 años y de ascendencia india Prima.

In [17]:
import pandas as pd
import sklearn.datasets as datasets

data = datasets.fetch_openml(name='diabetes', version=1)
df = pd.DataFrame(data['data'], columns=data['feature_names'])
target = data['target']

Veamos como luce el conjunto de datos

In [15]:
df.head(10)

,preg,plas,pres,skin,insu,mass,pedi,age
0,6.0,148.0,72.0,35.0,0.0,33.6,0.627,50.0
1,1.0,85.0,66.0,29.0,0.0,26.6,0.351,31.0
2,8.0,183.0,64.0,0.0,0.0,23.3,0.672,32.0
3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21.0
4,0.0,137.0,40.0,35.0,168.0,43.1,2.288,33.0
5,5.0,116.0,74.0,0.0,0.0,25.6,0.201,30.0
6,3.0,78.0,50.0,32.0,88.0,31.0,0.248,26.0
7,10.0,115.0,0.0,0.0,0.0,35.3,0.134,29.0
8,2.0,197.0,70.0,45.0,543.0,30.5,0.158,53.0
9,8.0,125.0,96.0,0.0,0.0,0.0,0.232,54.0


Entrenaremos un simple modelo para ilustrar el caso. No prestaremos demasiada atención en la ingeniería de predictores, pero si nos ocuparemos de trabajar con los valores faltantes. En particular, este conjunto de datos tiene varios predictores con valores nulos codificados como zeros (0). En nuestro caso, optaremos por remplazar estos valores por la media. Esta elección ha sido realizada de forma trivial.

In [18]:
from sklearn.impute import SimpleImputer

fill_values = SimpleImputer(missing_values=0, strategy="mean")
df = fill_values.fit_transform(df)

Entrenaremos nuestro modelo y lo evaluamos. En este caso, utilizaremos un simple `RandomForestClassifier`, provisto por la librería `scikit-learn`.

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, target, test_size=0.2, random_state=42, stratify=target)

In [21]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=10)
model.fit(X_train,y_train)

RandomForestClassifier(random_state=10)

In [24]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

                 precision    recall  f1-score   support

tested_negative       0.77      0.84      0.80       100
tested_positive       0.64      0.54      0.59        54

       accuracy                           0.73       154
      macro avg       0.71      0.69      0.69       154
   weighted avg       0.73      0.73      0.73       154

